# Profiling computation of derivatives with numpy vs dask vs xarray

### Author 
 - Julien Le Sommer, CNRS

### Context
 - April 2016, preparatory work for designing modelgrids grids in [oocgcm](https://github.com/lesommer/oocgcm)
 
### Purpose
 - compare the execution speed for computing derivatives with several types and combination of arrays 

### Types of array to consider 
 - **np** : numpy arrays
 - **da** : dask arrays build from netCDF4.Dataset without chunks 
 - **xa** : xarray arrays without chunks 
 - **dachk** : dask arrays build from netCDF4.Dataset with chunks
 - **danpchk** : dask arrays built from numpy array with chunks
 - **xachk** : xarray arrays with chunks

### Combination to test
 array tgrid / array e1u : 
  - [x]  np/np
  - [x] danpchk /danpchk
  - [x] dachk/dachk
  - [x] dachk/danpchk
  - [x] xachk /xachk


### Useful resource
http://nbviewer.jupyter.org/gist/rabernat/3e2c3645d4666674cf2b

### Modules

In [1]:
#- import
import numpy as np
from netCDF4 import Dataset
import dask.array as da
import xarray as xr
from contextlib import contextmanager
import time

In [2]:
#- print versions
import dask 
print('dask version : ' + dask.__version__)
print('xarray version : ' + xr.__version__)

dask version : 0.8.1
xarray version : 0.7.1


In [3]:
from dask.cache import Cache
cache = Cache(2e9)  # Leverage two gigabytes of memory
cache.register()    # Turn cache on globally

### Define the arrays

#### Size of chunks 

In [4]:
# for 2D-arrays
chunks2d = (1727,2711)
xr_chunks2d = {'x': chunks2d[-1], 'y': chunks2d[-2]}

# for 2D-arrays
#chunks3d = (50,378,516)
chunks3d = (25,756,1032)
xr_chunks3d_nodep = {'x': chunks3d[-1], 'y': chunks3d[-2]}
xr_chunks3d = {'x': chunks3d[-1], 'y': chunks3d[-2],'deptht':chunks3d[-3]}

#### Location of netcdf files

In [5]:
#- medium size 3D dataset from NATL60
#  array shape is (300,756,1032)
file_3d_gridt = "/Users/lesommer/data/NATL60/NATL60-MJM155-S/1d/2008/NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc"
varname_3d = "votemper"
file_3d_coord = "/Users/lesommer/data/NATL60/NATL60-I/BOXES/NATL60LMX_coordinates_v4.nc"

In [6]:
#- Medium size 2D dataset from NATL60 
#  array shape is (1,3454,5422)
file_2d_gridt = "/Users/lesommer/data/NATL60/NATL60-MJM155-S/1d/2008/NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc"
varname_2d = "vosigma0"
file_2d_coord = "/Users/lesommer/data/NATL60/NATL60-I/NATL60_coordinates_v4.nc"

#### Buid the arrays

In [7]:
verbose = True
def verbose_loading(filename,varname,array):
    print('loading from file ' + filename.split('/')[-1])
    print('    variable : ' + varname +  ', returned shape : ' + str(array.shape) )

In [8]:
#- numpy : np
def load_numpy(filename,varname,it=None):
    if it is None:
        out =  Dataset(filename).variables[varname][:].squeeze() 
    else: 
        out =  Dataset(filename).variables[varname][it][:].squeeze()
    if verbose: verbose_loading(filename,varname,out)
    return out

def get_array2d_tgrid_numpy():
    array_tgrid = load_numpy(file_2d_gridt,varname_2d,it=0)
    return array_tgrid

def get_array2d_e1u_numpy():
    array_e1u = load_numpy(file_2d_coord,"e1u")
    return array_e1u

def get_array3d_tgrid_numpy():
    array_tgrid = load_numpy(file_3d_gridt,varname_3d,it=0)
    return array_tgrid

def get_array3d_e1u_numpy():
    array_e1u = load_numpy(file_3d_coord,"e1u")
    return array_e1u

In [9]:
#- dask from numpy : danpchk
def load_danpchk(filename,varname,chunks,it=None):
    if it is None:
        d = Dataset(filename).variables[varname][:].squeeze()
    else :
        d = Dataset(filename).variables[varname][it][:].squeeze()
    array = da.from_array(np.array(d), chunks=chunks)
    if verbose: verbose_loading(filename,varname,array)
    return array 

def get_array2d_tgrid_danpchk():
    array_tgrid = load_danpchk(file_2d_gridt,varname_2d,chunks2d,it=0)
    return array_tgrid

def get_array2d_e1u_danpchk():
    array_e1u = load_danpchk(file_2d_coord,"e1u",chunks2d)
    return array_e1u

def get_array3d_tgrid_danpchk():
    array_tgrid = load_danpchk(file_3d_gridt,varname_3d,chunks3d,it=0)
    return array_tgrid

def get_array3d_e1u_danpchk():
    array_e1u = load_danpchk(file_3d_coord,"e1u",chunks3d)
    return array_e1u


In [10]:
#- dask from netcdf :  dachk
def load_dachk(filename,varname,chunks,it=None):
    d = Dataset(filename).variables[varname]
    if it is None:
        array = da.from_array(d, chunks=chunks)
    else :
        array = da.from_array(d, chunks=(1,)+ chunks)[it]
    if verbose: verbose_loading(filename,varname,array)
    return array 

def get_array2d_tgrid_dachk():
    array_tgrid = load_dachk(file_2d_gridt,varname_2d,chunks2d,it=0)
    return array_tgrid

def get_array2d_e1u_dachk():
    array_e1u = load_dachk(file_2d_coord,"e1u",chunks2d)
    return array_e1u

def get_array3d_tgrid_dachk():
    array_tgrid = load_dachk(file_3d_gridt,varname_3d,chunks3d,it=0)
    return array_tgrid

def get_array3d_e1u_dachk():
    array_e1u = load_dachk(file_3d_coord,"e1u",chunks3d)
    return array_e1u


In [11]:
#- xarray with chunks :  xachk
def load_xachk(filename,varname,xr_chunks,it=None):
    ds = xr.open_dataset(filename,chunks=xr_chunks)
    if it is None:
       array = ds.variables[varname]
    else:
        array = ds.variables[varname][it]
    if verbose: verbose_loading(filename,varname,array)
    return array.chunk(lock=True)

def get_array2d_tgrid_xachk():
    array_tgrid = load_xachk(file_2d_gridt,varname_2d,xr_chunks2d,it=0)
    return array_tgrid

def get_array2d_e1u_xachk():
    array_e1u = load_xachk(file_2d_coord,"e1u",xr_chunks2d)
    return array_e1u

def get_array3d_tgrid_xachk():
    array_tgrid = load_xachk(file_3d_gridt,varname_3d,xr_chunks3d,it=0)
    return array_tgrid

def get_array3d_e1u_xachk():
    array_e1u = load_xachk(file_3d_coord,"e1u",xr_chunks3d_nodep)
    return array_e1u

### Define the function that compute the x-derivative

In [12]:
#- Simple x-derivative function 
def derivative_np(array_tgrid,array_e1u):
    return ( np.roll(array_tgrid,-1,axis=-1) - array_tgrid ) / array_e1u

def derivative_da(array_tgrid,array_e1u):
    # using map_overlap with ghosts, as described in http://dask.pydata.org/en/latest/ghost.html
    di = lambda t: (np.roll(t,-1,axis=-1) - t) 
    depth = {-1: 1, -2: 0, -3:0}
    d = array_tgrid.map_overlap(di,depth=depth,boundary=0)
    return np.asarray(  (d / array_e1u ).compute() )

def derivative_xa(array_tgrid,array_e1u):
    return np.asarray((array_tgrid.shift(x=-1) - array_tgrid) / array_e1u)# caution xarr.roll()==xarr !!!

def derivative_xa_as_da(array_tgrid,array_e1u):
    return derivative_da(array_tgrid.data,array_e1u.data)

### Profiling execution
#### Profiling tools

In [13]:
@contextmanager
def timeit_context(name):
    startTime = time.time()
    yield
    elapsedTime = time.time() - startTime
    print('{} takes {} ms'.format(name, int(elapsedTime * 1000))+'\n')

In [14]:
def gradient_profile_context(func_get_array_gridt,func_get_array_e1u,compute_func): 
    "Simple profiling function"
    with timeit_context('    loading tgrid array'):
        array_tgrid = func_get_array_gridt()
    with timeit_context('    loading e1u array'):
        array_e1u  = func_get_array_e1u()
    with timeit_context('computing x-derivative'):
        gx = compute_func(array_tgrid, array_e1u)
    print('The output array is a ' + str(type(gx)))

#### Profiling experiments

##### Tests in 2 dimensions

In [15]:
# numpy / numpy
gradient_profile_context(get_array2d_tgrid_numpy,get_array2d_e1u_numpy,derivative_np)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 251 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 219 ms

computing x-derivative takes 765 ms

The output array is a <class 'numpy.ma.core.MaskedArray'>


In [16]:
# dask (numpy with chunk) / dask (numpy with chunk) 
gradient_profile_context(get_array2d_tgrid_danpchk,get_array2d_e1u_danpchk,derivative_da)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 398 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 612 ms

computing x-derivative takes 206 ms

The output array is a <type 'numpy.ndarray'>


In [17]:
# dask (netcdf with chunk) / dask (netcdf with chunk) 
gradient_profile_context(get_array2d_tgrid_dachk,get_array2d_e1u_dachk,derivative_da)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 2 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 0 ms

computing x-derivative takes 878 ms

The output array is a <type 'numpy.ndarray'>


In [18]:
# dask (netcdf with chunk) / dask (numpy with chunk) 
gradient_profile_context(get_array2d_tgrid_dachk,get_array2d_e1u_danpchk,derivative_da)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 1 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 647 ms

computing x-derivative takes 635 ms

The output array is a <type 'numpy.ndarray'>


In [19]:
# xarray (with chunk) / xarray (with chunk) 
gradient_profile_context(get_array2d_tgrid_xachk,get_array2d_e1u_xachk,derivative_xa)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 17 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 13 ms

computing x-derivative takes 32261 ms

The output array is a <type 'numpy.ndarray'>


**observation #1** :  xarray with chunks is x50 slower that dask arrays in this example. 

In [20]:
# xarray (with chunk) / xarray (with chunk) (derivative as for dask array)
gradient_profile_context(get_array2d_tgrid_xachk,get_array2d_e1u_xachk,derivative_xa_as_da)

loading from file NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc
    variable : vosigma0, returned shape : (3454, 5422)
    loading tgrid array takes 13 ms

loading from file NATL60_coordinates_v4.nc
    variable : e1u, returned shape : (3454, 5422)
    loading e1u array takes 10 ms

computing x-derivative takes 151475 ms

The output array is a <type 'numpy.ndarray'>


**observation #2** :  if we compute the derivative with xarray.data with the same method than for dask array, the execution time is even slower (x200)

##### Tests in 3 dimensions

In [21]:
# numpy / numpy
gradient_profile_context(get_array3d_tgrid_numpy,get_array3d_e1u_numpy,derivative_np)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 3027 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 17 ms

computing x-derivative takes 9755 ms

The output array is a <class 'numpy.ma.core.MaskedArray'>


In [22]:
# dask (numpy with chunk) / dask (numpy with chunk) 
gradient_profile_context(get_array3d_tgrid_danpchk,get_array3d_e1u_danpchk,derivative_da)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 5265 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 28 ms

computing x-derivative takes 4329 ms

The output array is a <type 'numpy.ndarray'>


In [23]:
# dask (netcdf with chunk) / dask (netcdf with chunk) 
gradient_profile_context(get_array3d_tgrid_dachk,get_array3d_e1u_dachk,derivative_da)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 4 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 1 ms

computing x-derivative takes 7877 ms

The output array is a <type 'numpy.ndarray'>


In [24]:
# dask (netcdf with chunk) / dask (numpy with chunk) 
gradient_profile_context(get_array3d_tgrid_dachk,get_array3d_e1u_danpchk,derivative_da)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 2 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 29 ms

computing x-derivative takes 7042 ms

The output array is a <type 'numpy.ndarray'>


In [25]:
# xarray (with chunk) / xarray (with chunk) 
gradient_profile_context(get_array3d_tgrid_xachk,get_array3d_e1u_xachk,derivative_xa)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 25 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 14 ms

computing x-derivative takes 6383 ms

The output array is a <type 'numpy.ndarray'>


In [26]:
# xarray (with chunk) / xarray (with chunk) (derivative as for dask array)
gradient_profile_context(get_array3d_tgrid_xachk,get_array3d_e1u_xachk,derivative_xa_as_da)

loading from file NATL60LMX-MJM155_y2008m03d15.1d_gridT.nc
    variable : votemper, returned shape : (300, 756, 1032)
    loading tgrid array takes 19 ms

loading from file NATL60LMX_coordinates_v4.nc
    variable : e1u, returned shape : (756, 1032)
    loading e1u array takes 19 ms

computing x-derivative takes 8596 ms

The output array is a <type 'numpy.ndarray'>


**observation #3 : ** For 3D arrays, we recover performances comparible to dask array with xarray. 